In [ ]:
import requests
import csv
import time

def retrieve_articles(api_key, query, num_articles):
    articles = []
    page = 0
    while len(articles) < num_articles:
        url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
        params = {
            "q": query,
            "api-key": api_key,
            "page": page
        }

        response = requests.get(url, params=params)

        if response.status_code == 200:
            results = response.json()["response"]["docs"]
            if len(results) == 0:
                break
            articles.extend(results)
            page += 1
        elif response.status_code == 429:
            print("Rate limit exceeded. Waiting and retrying...")
            time.sleep(60)  # Wait for 60 seconds before retrying
        else:
            print("Error occurred while retrieving articles:", response.status_code)
            break

    return articles[:num_articles]

def save_to_csv(articles, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['headline', 'web_url', 'pub_date', 'abstract']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for article in articles:
            writer.writerow({'headline': article["headline"]["main"],
                             'web_url': article["web_url"],
                             'pub_date': article["pub_date"],
                             'abstract': article["abstract"]})

# Replace 'YOUR_API_KEY' with your actual New York Times API key
api_key = 'CZm0kVm8LZoyPkhwSm42ef74Ry15ywGG'

# Replace 'YOUR_QUERY' with your search query
query = 'YOUR_QUERY'

# Number of articles you want to fetch
num_articles = 4000

articles = retrieve_articles(api_key, query, num_articles)
if articles:
    csv_filename = 'nytimes_articles.csv'
    save_to_csv(articles, csv_filename)
    print("Articles saved to", csv_filename, "successfully!")
else:
    print("No articles found.")
